In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
patH = input('작업폴더 경로를 입력해주세요 : ')     # /content/gdrive/MyDrive/Project/image/doro_contest
%cd {patH}

In [ ]:
%cd {patH}/image_labels

In [ ]:
from glob import glob
zip_list = glob('*.zip')
zip_list = sorted(zip_list)

In [ ]:
len(zip_list)

60

In [ ]:
import zipfile

for zip_name in zip_list:



  with zipfile.ZipFile(f'{patH}/image_labels/{zip_name}', 'r') as zip_ref:
      zip_ref.extractall(f'{patH}/dataset')

## yolo5 clone

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

yolov5에 필요한 패키지 다운

In [ ]:
%cd /content/yolov5
!pip install -r requirements.txt

/content/yolov5
     |████████████████████████████████| 596 kB 13.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


data.yaml에 있는 내용 보기

 /content/yolov5/data/ 여기에 data.yaml 업로드하기

In [ ]:
%cat /content/yolov5/data/data.yaml

train: ../train/images
val: ../valid/images

nc: 30
names: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '36', '37', '38']

train, validation 이미지 경로를 txt파일로 저장

In [ ]:
train_img_list = os.listdir(f'{patH}/dataset/train/images')
for i in range(len(train_img_list)) : 
  train_img_list[i] = patH + '/dataset/train/images/' + train_img_list[i]
val_img_list = os.listdir(f'{patH}/dataset/valid/images')
for i in range(len(val_img_list)) : 
  val_img_list[i] = patH + '/dataset/valid/images/' + val_img_list[i]

/


In [ ]:
with open(f'{patH}/dataset/train.txt','w') as f : 
  f.write('\n'.join(train_img_list)+'\n')

with open(f'{patH}/dataset/val.txt', 'w') as f : 
  f.write('\n'.join(val_img_list)+'\n')

## data.yaml 파일 수정

In [ ]:
import yaml

with open('/content/yolov5/data/data.yaml','r') as f : 
  data = yaml.full_load(f)   # json 부르는 형식이랑 같음

print(data)

data['train'] = patH + '/dataset/train.txt'
data['val'] = patH + '/dataset/val.txt'

with open('/content/yolov5/data/data.yaml','w') as f : 
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 30, 'names': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '36', '37', '38']}
{'train': '/content/gdrive/MyDrive/Project/image/인공지능_학습용_데이터_해커톤_대회/dataset/train.txt', 'val': '/content/gdrive/MyDrive/Project/image/인공지능_학습용_데이터_해커톤_대회/dataset/val.txt', 'nc': 30, 'names': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '36', '37', '38']}


## train

[train code 실행 화면 설명]

* pretrained 모델 받은 후
* train 시작
  * 한 에포크 당 출력되는 수치 설명
  * Class : all 이후 수치들 순서대로 설명
  * 이미지 개수, 레이블 개수, precision, recall, map0.5, map0.95
* training의 결과는 yolov5 > runs > train 에 --name 이후 지정했던 이름대로 저장됨
  * weight
  * hyperparameter
  * test_batch0_gt.jpg : 실제 참 값
  * test batch 실시간 표현  : test_batch0_pred.jpg 열어보면 알 수 있다
  * training_batch0.jpg ~ 도 batch를 보여줌
* 실행 마지막에는 weight 저장 위치 출력 : last_~.pt
* results.png : 학습 결과 그래프 이미지 저장

In [ ]:
%cd /content/yolov5

!python train.py --img 416 --batch 80 --epochs 180 --data ./data/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name road_yolov5s_results
# cfg는 모델의 구조를 나타냄. 이건 yolov5 > models 에 정의되어 있다(크기순으로 yolov5l.yaml > yolov5m.yaml > yolov5s.yaml > yolov5x.yaml)
# weights는 pretrained된 것 사용. yolov5s.pt를 다운로드 받아서 사용

## 텐서보드로 학습결과 보기

* content/yolov5/runs/train/road_yolov5s_results/weights  경로에서 last.pt와 best.pt 파일 다운로드
* 디렉토리에 best64_180.pt와 last64_180.pt파일로 저장해놓음

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

### Check Our Detecting Result

In [ ]:
val_imgs = glob(patH+'/test_input/*.jpg')

In [ ]:
Image = cv2.imread(val_imgs[0])
Image = cv2.cvtColor(Image, cv2.COLOR_BGR2RGB)
results = model(Image)

In [ ]:
# 결과! 
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.figure(figsize=(50,50))
plt.show()